In [ ]:
import base64
import html
import io
import time

from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

In [ ]:
start_input()
label_html = 'Capturing...'
img_data = ''

while True:
  capture_start = time.time()  
  js_reply = take_photo(label_html, img_data)
  capture_end = time.time()
  if not js_reply:
    break

In [ ]:
from google.colab import drive
from google.colab import files
from google.colab import auth
from google.cloud import storage

drive.mount('/content/gdrive')
!ls /content/gdrive/

auth.authenticate_user()

project_id = 'charming-storm-291907'
!gcloud config set project {project_id}
!gsutil ls

bucket_name = 'aiot-storage'

In [ ]:
!git clone https://github.com/ultralytics/yolov3.git  # clone
%cd yolov3

from sys import platform
import argparse

from models import *
from utils.datasets import *
from utils.utils import *
from absl.flags import FLAGS
from absl import app, flags, logging

parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, default='cfg/yolov3-spp.cfg', help='*.cfg path')
parser.add_argument('--names', type=str, default='data/coco.names', help='*.names path')
parser.add_argument('--weights', type=str, default='weights/yolov3-spp-ultralytics.pt', help='weights path')
parser.add_argument('--img-size', type=int, default=416, help='inference size (pixels)')
parser.add_argument('--conf-thres', type=float, default=0.3, help='object confidence threshold')
parser.add_argument('--iou-thres', type=float, default=0.6, help='IOU threshold for NMS')
parser.add_argument('--classes', nargs='+', type=int, help='filter by class')
parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
opt = parser.parse_args(args = [])

# Initialize
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Initialize model
model = Darknet(opt.cfg, opt.img_size)

# Load weights
attempt_download(opt.weights)
if opt.weights.endswith('.pt'):  # pytorch format
    model.load_state_dict(torch.load(opt.weights, map_location=device)['model'])
else:  # darknet format
    load_darknet_weights(model, opt.weights)

model.to(device).eval();

# Get names and colors
names = load_classes(opt.names)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

In [ ]:
from datetime import datetime
import os

now = datetime.now()
fsnow = str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour + 9) + '_' + str(now.minute) + '_' + str(now.second)

dirc = '/content/gdrive/My Drive/record_files2/images/' + fsnow
isFile = os.path.isdir(dirc)

if not isFile:
    os.mkdir(dirc)

dirc2 = '/content/gdrive/My Drive/record_files2/gray_images/' + fsnow
isFile2 = os.path.isdir(dirc2)

if not isFile2:
    os.mkdir(dirc2)

name = fsnow + '/'
storage_client = storage.Client(project_id)
bucket = storage_client.bucket(bucket_name)
stats = storage.Blob(bucket=bucket, name=name).exists(storage_client)

#if (stats == False):
!gsutil mb gs://{bucket_name}/record_files2/images/{fsnow}/
!gsutil mb gs://{bucket_name}/record_files2/gray_images/{fsnow}/

In [ ]:
def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam
    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    return image_array

def get_drawing_array(image_array): 
    """
    input: 
          image_array: image array RGB size 512 x 512 from webcam
    output: 
          drawing_array: image RGBA size 512 x 512 only contain bounding box and text, 
                              channel A value = 255 if the pixel contains drawing properties (lines, text) 
                              else channel A value = 0
    """
    name_l = []
    drawing_array = np.zeros([512,512,4], dtype=np.uint8)
    img = letterbox(image_array, new_shape=opt.img_size)[0]

    img = img.transpose(2, 0, 1)
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img)[0]
    # Apply NMS
    pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)

    # Process detections
    det = pred[0]
    str_obj = ''
    if det is not None and len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], image_array.shape).round()

        # Write results
        for *xyxy, conf, cls in det:
            label = '%s %.2f' % (names[int(cls)], conf)
            plot_one_box(xyxy, drawing_array, label=label, color=colors[int(cls)])
            name_l.append(names[int(cls)])

    drawing_array[:,:,3] = (drawing_array.max(axis = 2) > 0 ).astype(int) * 255    
    return drawing_array, name_l

def drawing_array_to_bytes(drawing_array):
    """
    input: 
          drawing_array: image RGBA size 512 x 512 
                              contain bounding box and text from yolo prediction, 
                              channel A value = 255 if the pixel contains drawing properties (lines, text) 
                              else channel A value = 0
    output: 
          drawing_b64: string, encoded from drawing_array
    """

    drawing_PIL = Image.fromarray(drawing_array, 'RGBA')
    iobuf = io.BytesIO()
    drawing_PIL.save(iobuf, format='png')
    drawing_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))
    return drawing_bytes

In [ ]:
import os
from scipy import ndimage
from datetime import datetime
from matplotlib import pyplot as plt
from io import BytesIO
from base64 import b64decode
import operator

start_input()
label_html = 'Capturing...'
img_data = ''
name_l = []

now = datetime.now()
snow = str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour + 9) + '_' + str(now.minute) + '_' + str(now.second)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('/content/gdrive/My Drive/record_files2/' + snow + '.mp4', fourcc, 1.0, (512, 512))
filename = snow + '.mp4'

while True:
    now = datetime.now()
    snow = str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour + 9) + '_' + str(now.minute) + '_' + str(now.second)
    js_reply = take_photo(label_html, img_data)

    if not js_reply:
        break

    image = js_reply_to_image(js_reply)
    
    if 'person' in name_l:

        drawing_array, name_l = get_drawing_array(image)
        drawing_bytes = drawing_array_to_bytes(drawing_array)
        img_data = drawing_bytes
        ed_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        drawing_array = cv2.cvtColor(drawing_array, cv2.COLOR_RGBA2BGR)

        cv2.imwrite('/content/gdrive/My Drive/record_files2/images/' + fsnow + '/' + snow + '.png', ed_image)

        x = []
        y = []
        img = drawing_array[:,:,0] # convert to 2D array
        row, col = img.shape
        for i in range(row):
            for j in range(col):
                if img[i,j] != 0:
                    x.append(i) # get x indices
                    y.append(j) # get y indices

        for i in range(len(x)):
            image[x[i],y[i]] = (0, 0, 0)

        yolo_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        writer.write(yolo_img + drawing_array)
    
    else:
        
        image2 = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        drawing_array, name_l = get_drawing_array(image)
        drawing_bytes = drawing_array_to_bytes(drawing_array)
        img_data = drawing_bytes
        cv2.imwrite('/content/gdrive/My Drive/record_files2/gray_images/' + fsnow + '/' + snow + '.png', image2)

        drawing_array = cv2.cvtColor(drawing_array, cv2.COLOR_RGBA2BGR)

        x = []
        y = []
        img = drawing_array[:,:,0] # convert to 2D array
        row, col = img.shape
        for i in range(row):
            for j in range(col):
                if img[i,j] != 0:
                    x.append(i) # get x indices
                    y.append(j) # get y indices

        for i in range(len(x)):
            image[x[i],y[i]] = (0, 0, 0)

        yolo_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        writer.write(yolo_img + drawing_array)

        inimg = plt.imread('/content/gdrive/My Drive/record_files2/gray_images/' + fsnow + '/' + snow + '.png')
        inimg = np.uint8(255*inimg)

        # =============================================================================
        # # edge detection
        # =============================================================================
        x,y = inimg.shape
        edge = np.zeros(256)
        sobel1 = 1/4*np.array([[1,0,-1],
                              [2,0,-2],
                              [1,0,-1]])

        sobel2 = 1/4*np.array([[-1,-2,-1],
                              [0,0,0],
                              [1,2,1]])

        inimg_x = ndimage.convolve(inimg[:,:], sobel1)
        inimg_y = ndimage.convolve(inimg[:,:], sobel2)

        #x gradient 
        inimg_x = np.uint8(inimg_x)

        #y gradient 
        inimg_y = np.uint8(inimg_y)

        # sobel_inimg = abs(inimg_x) + abs(inimg_y)
        sobel_inimg = (inimg_x**2 + inimg_y**2)**0.5
        sobel_outimg = np.zeros([x,y])
        threshold = 5

        for i in range(x):
            for j in range(y):
                if (sobel_inimg[i,j] >= threshold):
                    sobel_outimg[i,j] = 0
                else:
                    sobel_outimg[i,j] = 255

        sobel_outimg = np.float32(sobel_outimg)
        cv2.imwrite('/content/gdrive/My Drive/record_files2/images/' + fsnow +'/' + snow + '.png', sobel_outimg)   

image_folder = '/content/gdrive/My Drive/record_files2/images' +'/' + fsnow
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter('/content/gdrive/My Drive/record_files2/ed_' + filename, cv2.VideoWriter_fourcc(*'mp4v'), 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

writer.release()
video.release()
cv2.destroyAllWindows()

!gsutil cp -r /content/gdrive/My\ Drive/record_files2/ed_{filename} gs://{bucket_name}/record_files2/      # 구글 드라이브에 있는 파일을 구글클라우드에 저장